In [11]:
# requires csg2step and OpenSCAD
# https://github.com/gega/csg2stp

In [12]:
import numpy as np, pandas as pd, shapely
import design_parameters as dp
import os
beampipe0=dp.beampipe0

In [13]:

def mkdir(path):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
mkdir("csg")
mkdir("openscad")

In [14]:
class Component():
    def __init__(self,z_offset, thickness, name):
        self.z_offset=z_offset
        self.thickness=thickness
        self.name=name
z=0
components={}
for name, thickness in [("absorber", 1.61), ("airgap1", 0.02),("cover", 0.04),                     
                        ("foil1", 0.015),("scint", 0.3), ("foil2", 0.015), ("pcb", 0.08), 
                        ("airgap2", 0.02)]:
    components[name]=Component(z, thickness, name)
    z+=thickness
print(z)

2.1


In [15]:
#first create the generic layers:
cm=10
det_height=dp.det_height
det_width=dp.det_width
det_gap=dp.det_gap
for compname in "absorber cover foil1 foil2 pcb".split():
    comp = components[compname]
    scad_file=f"openscad/all_layers_{compname}.scad"
    csg_file=f"all_layers_{compname}.csg"
    with open(scad_file, "w") as f:
        print("include <generic_layer.scad>",file=f)
        
        print("union(){", file=f)
        for layer in range(54):
            z=beampipe0.getZ(layer)+comp.z_offset
            holeR=beampipe0.holeR(layer)
            holeX=beampipe0.holeX(layer)
            thickness=comp.thickness
            print(f"generic_layer({z*cm}, {holeX*cm}, {holeR*cm}, {thickness*cm}, det_height={det_height*cm},"+ \
                  f"det_width={det_width*cm}, det_gap={det_gap*cm});", file=f)
            
        print("}", file=f)
    os.system(f'OpenSCAD  -o {csg_file} {scad_file}') 
    
    #os.system(f'mv {csg_file} csg/{csg_file}')    
    
        
    

In [16]:


#next create the scintillator cells (based on the database of all layers)
#first read in the database of scintillator positions/shapes
import pickle
with open("all_layers.pkl", "rb") as file:
    all_layers=pickle.load(file)
#print(all_layers)
scad_file="openscad/all_layers_scint.scad"
with open(scad_file, "w") as f:
    #print("include <scint_lib.scad>",file=f)
    for layer in range(54):
        
        for side in "LR":
            print(f"// begin layer {layer}, side {side}", file=f)
            df=all_layers[(layer, side)]
            z=beampipe0.getZ(layer)+components['scint'].z_offset

            thickness=components['scint'].thickness
            for cell in range(len(df.scint_boundsx)):
                if df.deadzone[cell]:
                    continue
                x,y=df.scint_boundsx[cell], df.scint_boundsy[cell]

                xystr="[" + (", ".join(['[%.2f, %.2f]'%(x[i]*cm, y[i]*cm) for i in range(len(x))])) + "]"

                #print(xstr)
                command=f"translate([0,0,{z*cm}]) linear_extrude({thickness*cm}) polygon(points={xystr});"

                print(command, file=f)
            print(f"// end layer {layer}, side {side}", file=f)
           
    os.system(f'OpenSCAD  -o {csg_file} {scad_file}') 
